# <h1 align="center"> THEME 6 - Résolution analytique d'équations différentielles</h1>

### 🎯 Objectifs
- Résoudre des équations différentielles analytiquement

### 📚 Notions 

- [Exemple 1](#ex1):
- [Exemple 2](#ex2):
- [Exemple 3](#ex3):

Un [lexique](#lexique) avec l'ensemble des fonctions qui ont été vues est disponible à la fin du notebook.

### 🧰 Librairies

- **Sympy**: est une librairie Python pour le calcul symbolique. Elle vise à devenir un système de calcul formel (CAS) complet tout en gardant le code aussi simple que possible afin d'être compréhensible et facilement extensible.

### 🔗 Référence

- [Documentation Sympy](https://docs.sympy.org/latest/reference/index.html#reference)

### ⚙️ Installation

`pip install sympy`

## <a name="ex1"><h2 align="center"> Exemple 1 - Réaction Non-Linéaire </h2></a>

### 📝 Contexte

Soit la réaction suivante:

$$
2 A \rightarrow B
$$

Son équation est une ODE non-linéaire:

$$
\frac{d C_{A}}{d t}=-k C_{A}^{2}
$$

### 🧪 Paramètres
 
Donnée:
- $k=2 \, L.mol^{-1}.min^{-1}$

Condition initiale:
- $C_{A0}=1 \, mol.L^{-1}$

### ⭐ Objectif

Résoudre l'ODE numériquement et tracer la concentration de A entre t=0 et t=10 min avec 26 points.

### 💻 Code

### 💡 Astuces

Sympy est une librairie très complète et qui possède un grand nombre de fonctions pour la résolution de toute sorte d'équations. Pour ce thème, le sous-module [ODE](https://docs.sympy.org/latest/modules/solvers/ode.html) de Sympy sera principalement utilisé.

Tout calcul symbolique avec Sympy se fait avec 3 types de données:
1. Des variables (ou symboles) définis avec `Symbol` ou `symbols`.
2. Des fonctions défénies avec `Function`.
3. Des équations (ou expressions) définies avec `Eq`.

Pour commencer, on définit notre équation non-linéaire sans utiliser de paramètres numériques. 

In [ ]:
import sympy as sym  # Importer la librairie sympy
import matplotlib.pyplot as plt
import numpy as np

sym.init_printing()  # Initialiser le systeme d'affichage LaTeX

# Déclarations
# ---------------------------------------------------
t, k = sym.symbols("t k")  # Variables t et k
C = sym.Function("C")(t)  # Fonction C qui dépend de t
dCdt = C.diff(t)  # Dérivée de C par rapport à t
expr = sym.Eq(dCdt, -k * C**2)

# Affichage de l'équation (uniquement dans un notebook)
# ---------------------------------------------------
expr

Une fois l'équation différentielle définie, on peut la résoudre avec la fonction `dsolve`.

In [ ]:
sol_analytique = sym.dsolve(expr)
sol_analytique

On peut remplacer une variable avec valeur numérique avec `subs`.

In [ ]:
sol_analytique.subs(k, 2)

Pour résoudre l'équation différentielle avec des conditions initiales, on peut passer un dictionnaire à l'argument `ics` de la fonction `dsolve`.

In [ ]:
k = 2  # Définir la valeur de k
expr = sym.Eq(dCdt, -k * C**2)  # Redéfinir l'équation avec la nouvelle valeur de k

# ics prend un dictionnaire avec comme clé la fonction avec sa variable dépendante définie numériquement et comme valeur
# la condition initiale
sol_condinit = sym.dsolve(expr, ics={C.subs(t, 0): 1})  # C_A0 = 1
sol_condinit

On peut évaluer notre solution à un instant t en remplaçant la variable `t` par sa valeur numérique. 

In [ ]:
sol_condinit.subs(t, 3)

Noter que la solution est une équation, c'est à dire qu'elle contient une partie `lhs` dépendante de la variable d'état (ici `t`) et une partie `rhs` indépendante de cette variable.

Pour pouvoir évaluer la solution sur un ensemble de valeurs de `t` (comme un vecteur Numpy) il faut la transformer en fonction. Pour cela on utilise la fonction `lambdify` sur la partie `rhs` de la solution.

In [ ]:
from sympy.utilities.lambdify import lambdify

# Création d'une fonction qui prend en paramètre t et renvoie la valeur de la solution
f = lambdify(t, sol_condinit.rhs)
vec_t = np.linspace(0, 10, 100)

# La fonction peut être directement évaluée sur un vecteur Numpy
plt.plot(vec_t, f(vec_t))
plt.show()

## <a name="lexique"><h2 align="center"> Lexique </h2></a>

### 📚 Terminologie

### ✔️ Vu dans l'exemple 1